In [1]:
import pandas as pd

df = pd.read_csv('data/realty_data.csv', encoding='utf-8').drop(columns='period')
df['product_name'] = df.product_name.str.replace(' ', ' ')
rs_coord = (55.7539, 37.6208)

df['rs_dist'] = (df['lat'] - rs_coord[0]) ** 2 + (df['lon'] - rs_coord[1]) ** 2
df

,product_name,price,postcode,address_name,lat,lon,object_type,total_square,rooms,floor,city,settlement,district,area,description,source,rs_dist
0,"3-комнатная, 137 м²",63000000,127473.0,"2-й Щемиловский переулок, 5а",55.778894,37.608844,Квартира,137.0,3.0,6.0,Москва,NaN,Тверской район,NaN,Просторная квартира свободной планировки с пан...,ЦИАН,0.000768
1,"Студия, 16,7 м²",3250000,108815.0,"Харлампиева, 46",55.551025,37.313054,Квартира,16.7,NaN,1.0,Москва,NaN,Филимонковское поселение,NaN,ВНИМАНИЕ! ОЧЕНЬ ПРИВЛЕКАТЕЛЬНОЕ ПРЕ...,Домклик,0.135866
2,"3-комнатная, 76 м²",16004680,NaN,"ЖК Прокшино, 8 к4",55.594802,37.431264,Квартира,76.0,3.0,6.0,Москва,NaN,Сосенское поселение,NaN,"Apт.1684018. 0,01% - гибкая ипотека! Воспользу...",Яндекс.Недвижимость,0.061236
3,"1-комнатная, 24 м²",7841776,NaN,"ЖК Прокшино, 6 к2",55.594332,37.428099,Квартира,24.0,1.0,10.0,Москва,NaN,Сосенское поселение,NaN,Продается однокомнатная квартира № 381 в новос...,Новострой-М,0.062596
4,"3-комнатная, 126 м²",120000000,121352.0,"Давыдковская, 18",55.721097,37.464342,Квартира,126.0,3.0,16.0,Москва,NaN,Фили-Давыдково район,NaN,Шикарное предложение!\nПродаётся трёхкомнатная...,Домклик,0.025555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98817,"2-комнатная, 42 м²",12183528,125212.0,"Кронштадтский бульвар, 9 к1/3",55.842935,37.487680,Квартира,42.0,2.0,8.0,Москва,NaN,Головинский район,NaN,Продаётся 2-комн.квартира площадью 42.00 кв.м ...,Яндекс.Недвижимость,0.025648
98818,"2-комнатная, 63,5 м²",34925000,129090.0,"Олимпийский проспект, вл10",55.777354,37.621747,Квартира,63.5,2.0,3.0,Москва,NaN,Мещанский район,NaN,С 1 октября повышение цен на 5% на апартаменты...,ЦИАН,0.000551
98819,"1-комнатная, 34,4 м²",10000000,117534.0,"Чертановская улица, 53 к1",55.602124,37.591573,Квартира,34.4,1.0,3.0,Москва,NaN,Чертаново Южное район,NaN,Продажа идеальной однокомнатной квартиры. Безу...,Домклик,0.023890
98820,"3-комнатная, 56 м²",15999000,109518.0,"1-й Грайвороновский проезд, 3",55.719431,37.724875,Квартира,56.0,3.0,11.0,Москва,NaN,Текстильщики район,NaN,"Новостройка! Евротрешка , две раздельная комна...",Домклик,0.012020


In [2]:
import joblib
from src.OLS import SMOLSWrapper, PandasSimpleImputer

reg_pipe = joblib.load('pipe.pkl')
clusterer = joblib.load('clusterer.pkl')
train_index = joblib.load('train_index.pkl')

In [3]:
cols = ['product_name', 'city', 'district', 'address_name', 'description']

x_text = df[cols].apply(lambda row: '. '.join(row.values.astype(str)), axis=1)
reg_features = ['total_square', 'rooms', 'floor', 'rs_dist']

reg_inputs = df[reg_features]
reg_inputs['cluster'] = clusterer.predict(df[['lat', 'lon']])

reg_preds = reg_pipe.predict(reg_inputs)

y_text = (df['price'] - reg_preds)

C:\Users\Anton\AppData\Local\Temp\ipykernel_28136\2697482820.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_inputs['cluster'] = clusterer.predict(df[['lat', 'lon']])


In [4]:
x_train = x_text[train_index].reset_index(drop=True)
y_train = y_text[train_index].to_numpy()

x_val = x_text[~x_text.index.isin(train_index)].reset_index(drop=True)
y_val = y_text[~x_text.index.isin(train_index)].to_numpy()

print(f'TRAIN SHAPE: {x_train.shape}\nVAL SHAPE: {x_val.shape}')

TRAIN SHAPE: (79057,)
VAL SHAPE: (19765,)


In [5]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, set_seed

set_seed(12)
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru", do_lower_case=True)

encoding_t = tokenizer(list(x_train), padding='max_length', truncation=True, max_length=200, return_tensors='pt')
encoding_v = tokenizer(list(x_val), padding='max_length', truncation=True, max_length=200, return_tensors='pt')

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape(-1, 1))
y_val = scaler.transform(y_val.reshape(-1, 1))

In [13]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 4


def create_dataloaders(inputs, masks, labels, batch_size, shuffle=False):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks).float()
    labels_tensor = torch.tensor(labels).float()
    dataset = TensorDataset(input_tensor, mask_tensor,
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            shuffle=shuffle)
    return dataloader


train_dataloader = create_dataloaders(encoding_t['input_ids'], encoding_t['attention_mask'],
                                      y_train, batch_size, shuffle=True)
val_dataloader = create_dataloaders(encoding_v['input_ids'], encoding_v['attention_mask'],
                                    y_val, batch_size)

C:\Users\Anton\AppData\Local\Temp\ipykernel_28136\2427315894.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(inputs)
C:\Users\Anton\AppData\Local\Temp\ipykernel_28136\2427315894.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tensor = torch.tensor(masks).float()


In [16]:
import torch.nn as nn
from transformers.modeling_utils import PreTrainedModel
from tqdm import tqdm

class SbertRegressor(PreTrainedModel):
    def __init__(self, interim_size=30, drop_rate=0.2):
        model_name = 'sberbank-ai/sbert_large_nlu_ru'
        cfg = AutoConfig.from_pretrained(model_name)
        super().__init__(cfg)
        D_in, D_out = 1024, 1

        self.base = AutoModel.from_pretrained(model_name)

        for param in self.base.parameters():
            param.requires_grad = False

        self.regressor = nn.Sequential(
            nn.Linear(D_in, interim_size),
            nn.Dropout(drop_rate),
            nn.ReLU(),
            nn.Linear(interim_size, D_out)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base(input_ids, attention_mask)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs

    def learnable_params(self):
        return self.regressor.parameters()


model = SbertRegressor()

In [17]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model = model.to(device)

Using GPU.


In [10]:
from torch.optim import AdamW

optimizer = AdamW(model.learnable_params())

In [11]:
loss_function = nn.MSELoss()

In [12]:
def train(model, optimizer, loss_function,
          train_dataloader, stop_step=5000, device='cuda'):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        if step > stop_step :
            break

        batch_inputs, batch_masks, batch_labels = \
            tuple(b.to(device) for b in batch)

        model.zero_grad()
        outputs = model(batch_inputs, batch_masks)
        loss = loss_function(outputs.squeeze(),
                             batch_labels.squeeze())


        loss.backward()
        optimizer.step()

    return model


model = train(model, optimizer, loss_function,
              train_dataloader)

 25%|██▌       | 5001/19765 [14:38<43:12,  5.70it/s]  


In [13]:
torch.save(model.state_dict(), 'model.pt')

In [18]:
model.load_state_dict(torch.load('model.pt'))
model.eval()

SbertRegressor(
  (base): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, 

In [19]:
import numpy as np
def predict(model, dataloader, out_scaler, device='cuda'):
    model.eval()
    output = []
    for batch in tqdm(dataloader):
        batch_inputs, batch_masks, _ = \
            tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs,
                            batch_masks).view(1, -1).tolist()[0]
    return pd.Series(out_scaler.inverse_transform(np.array(output).reshape(-1, 1)).reshape(-1))

In [20]:
residual_preds = predict(model, val_dataloader, scaler)

100%|██████████| 4942/4942 [13:38<00:00,  6.04it/s]


In [30]:
residual_preds = pd.Series(residual_preds.reshape(-1))

In [31]:
residual_preds.describe()

count    1.976500e+04
mean     2.736596e+05
std      8.263804e+05
min     -2.951442e+06
25%     -3.332121e+05
50%      3.619528e+05
75%      1.183413e+06
max      1.183413e+06
dtype: float64

In [52]:
joblib.dump(total_preds, 'preds.pkl')

['preds.pkl']